In [1]:
import xarray as xr
import glob
import intake
import numpy as np
import seaborn as sns
from scipy import stats
import os

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
import xesmf as xe




In [3]:
from plotting_maps.acs_plotting_maps import plot_acs_hazard_multi, plot_acs_hazard, plot_data, cmap_dict, regions_dict
from matplotlib import colors, cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from dask.diagnostics import ProgressBar
# ProgressBar().register()

import dask
from dask.distributed import Client
client = Client(threads_per_worker=2, n_workers=14)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 28,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42145,Workers: 14
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:33953,Total threads: 2
Dashboard: /proxy/44175/status,Memory: 8.94 GiB
Nanny: tcp://127.0.0.1:34625,


2025-09-25 21:07:24,997 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 27ee225f03468f980797dc4b2cb8214a initialized by task ('rechunk-merge-rechunk-transfer-c5b4c319b47137db1087680d4396bfbb', 0, 1, 3, 1, 1, 3) executed on worker tcp://127.0.0.1:34531
2025-09-25 21:07:25,048 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 7ac981e0e51af884e7559d01d4b99bae initialized by task ('rechunk-merge-rechunk-transfer-c5b4c319b47137db1087680d4396bfbb', 0, 0, 3, 1, 0, 3) executed on worker tcp://127.0.0.1:37847
2025-09-25 21:07:25,049 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 9d20abcd142ff4896017422d21a534ee initialized by task ('rechunk-merge-rechunk-transfer-c5b4c319b47137db1087680d4396bfbb', 0, 2, 5, 1, 2, 5) executed on worker tcp://127.0.0.1:40953
2025-09-25 21:07:25,156 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle d831ece09c09823e3f517264df536244 initialized by task ('rechunk-merge-rechunk-transfer-c5b4c319b47137db1087680d4396bfbb', 0, 

In [4]:
# for BARRA

In [5]:
# BARRA process has to be a bit different because the files are saved in single time slices, compared to single level with whole year


In [6]:
ds_target = xr.open_dataset("/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1_6hr_1979.nc")
ds_target


<xarray.Dataset> Size: 4GB
Dimensions:    (time: 1460, lat: 436, lon: 777)
Coordinates:
  * time       (time) datetime64[ns] 12kB 1979-01-01 ... 1979-12-31T18:00:00
  * lon        (lon) float64 6kB 88.04 88.19 88.34 88.5 ... 207.6 207.8 207.9
  * lat        (lat) float64 3kB -53.58 -53.42 -53.27 ... 13.32 13.48 13.63
    pressure   int64 8B ...
Data variables:
    windspeed  (time, lat, lon) float64 4GB ...
Attributes:
    long_name:      Air Temperature
    standard_name:  air_temperature
    units:          K
    cell_methods:   pressure: point time: point (interval: 6H) area: interpol...
    grid_mapping:   crs

In [7]:
lat_min, lat_max = (ds_target["lat"].values[0], ds_target["lat"].values[-1])
lon_min, lon_max = (ds_target["lon"].values[0], ds_target["lon"].values[-1])
lat_min, lat_max, lon_min, lon_max 

(-53.5755, 13.632, 88.0355, 207.9275)

In [ ]:
%%time


# testing
VARU = "wnd_ucmp"
VARV = "wnd_vcmp"

start_year = 1990
end_year = 2019

regridder=None
# target grid for regridding
ds_target = xr.open_dataset("/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1_6hr_1979.nc")

def _preprocess(ds):
    try:
        ds = ds.drop_vars( [ "forecast_period", "forecast_reference_time", "latitude_longitude"],)\
            .sel({"pressure":250}, method = "nearest")\
            .sel(latitude= slice(lat_min-0.2 , lat_max+0.2), longitude=slice(lon_min-0.2 , lon_max+0.2))
    except:
        ds = ds.drop_vars( ["latitude_longitude"],)\
            .sel({"pressure":250}, method = "nearest")\
            .sel(latitude= slice(lat_min-0.2 , lat_max+0.2), longitude=slice(lon_min-0.2 , lon_max+0.2))
    return ds     

max_retries = 30
retry_count = 0
while retry_count < max_retries:
    try:
        for year in np.arange(start_year, end_year+1):
            new_file = f"/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_{year}.nc"
            if os.path.exists(new_file):
                print(f"File '{new_file}' already exists.")
                continue
                
            # u component of wind
            print("filelist u...")
            filelist = glob.glob(f"/g/data/cj37/BARRA/BARRA_R/v1/analysis/prs/{VARU}/{year}/*/{VARU}-an-prs-PT0H-BARRA_R-v1-*T*00Z.sub.nc")
            filelist.sort()
        
            print("open_mfdataset u...")
            ds_u = xr.open_mfdataset(filelist, preprocess=_preprocess, combine="nested", concat_dim="time", parallel=True,)\
                    .chunk({"time":-1, "latitude":220, "longitude":112})\
                    .rename({f"{VARU}":"ua250", "latitude":"lat", "longitude":"lon"})
        
            # v component of wind
            print("filelist v...")
            filelist = glob.glob(f"/g/data/cj37/BARRA/BARRA_R/v1/analysis/prs/{VARV}/{year}/*/{VARV}-an-prs-PT0H-BARRA_R-v1-*T*00Z.sub.nc")
            filelist.sort()
        
            print("open_mfdataset v...")
            ds_v = xr.open_mfdataset(filelist, preprocess=_preprocess, combine="nested", concat_dim="time", parallel=True,)\
                    .chunk({"time":-1, "latitude":220, "longitude":112})\
                    .rename({f"{VARV}":"va250", "latitude":"lat", "longitude":"lon"})
        
            # calculate windspeed (lazy)
            print("calculate windspeed")
            ds_windspeed = xr.Dataset({"windspeed":xr.ufuncs.hypot(ds_u["ua250"], ds_v["va250"])}
                                     ).chunk({"time":120, "lat":-1, "lon":-1})
        
            ds_u.close()
            ds_v.close()
            
            if regridder is None:
                print("calculate regridder...")
                regridder = xe.Regridder(ds_windspeed, ds_target, "bilinear", unmapped_to_nan=True)
            print(f"regrid and compute {year}...")
            ds_regridded = regridder(ds_windspeed).compute()
            try:
                ds_regridded.to_netcdf(new_file)
                print(f"Saved {new_file}")
            except:
                print(f"cannot save {new_file}")
            ds_windspeed.close()
            ds_regridded.close()
        break  
    except:
        retry_count += 1
        print(f"Error. Retry count = {retry_count}")
else:
    print(f"Operation failed after {max_retries} attempts.")
        
            

File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1990.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1991.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1992.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1993.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1994.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1995.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1996.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_wind

HDF5-DIAG: Error detected in HDF5 (1.14.3) thread 0:
  #000: H5A.c line 1891 in H5Aiterate2(): invalid location identifier
    major: Invalid arguments to routine
    minor: Inappropriate type
  #001: H5VLint.c line 1733 in H5VL_vol_object(): invalid identifier
    major: Invalid arguments to routine
    minor: Inappropriate type
2025-09-25 21:04:13,051 - distributed.worker - ERROR - Compute Failed
Key:       open_dataset-ec5df51c-5afb-4b8c-8301-705a17d3a7ae
State:     executing
Task:  <Task 'open_dataset-ec5df51c-5afb-4b8c-8301-705a17d3a7ae' apply(...)>
Exception: 'RuntimeError("NetCDF: Can\'t open HDF5 attribute")'
Traceback: '  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/utils.py", line 77, in apply\n    return func(*args, **kwargs)\n  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/xarray/backends/api.py", line 670, in open_dataset\n    backend_ds = backend.open_dataset(\n  File "/g/data/

Error. Retry count = 1
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1990.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1991.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1992.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1993.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1994.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1995.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1996.nc' already exists.
File '/scratch/v46/gt3409

[gadi-cpu-bdw-0071.gadi.nci.org.au:509088] shmem: mmap: an error occurred while determining whether or not /jobfs/150845336.gadi-pbs/ompi.gadi-cpu-bdw-0071.12872/jf.0/3035037696/shared_mem_cuda_pool.gadi-cpu-bdw-0071 could be created.
[gadi-cpu-bdw-0071.gadi.nci.org.au:509088] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


regrid and compute 1998...
Saved /scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1998.nc
filelist u...
open_mfdataset u...
filelist v...
open_mfdataset v...
calculate windspeed
regrid and compute 1999...
Saved /scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1999.nc
filelist u...
open_mfdataset u...
filelist v...
open_mfdataset v...
calculate windspeed
regrid and compute 2000...
